In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding 
from datasets import load_metric
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
df = pd.read_csv("./resources/Trump_2016.csv")

In [6]:
df = df[df["source"].isin(['Twitter for iPhone', 'Twitter for Android'])]

In [7]:
df_train = df.iloc[:2600]
df_test = df.iloc[2600:]

In [8]:
df_train

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for Android,I will be on @FoxNews live with members of my...,01/01/2016 00:44,2108,6735,False,6.830000e+17
1,Twitter for iPhone,HAPPY NEW YEAR &amp; THANK YOU! https://t.co/Y...,01/01/2016 03:25,3460,8581,False,6.830000e+17
2,Twitter for iPhone,#HappyNewYearAmerica! https://t.co/EeQb8PDrUe,01/01/2016 05:18,3434,9143,False,6.830000e+17
3,Twitter for iPhone,Happy New Year from #MarALago! Thank you to my...,01/01/2016 06:07,1948,8258,False,6.830000e+17
4,Twitter for Android,Well the year has officially begun. I have man...,01/01/2016 23:00,2642,8495,False,6.830000e+17
...,...,...,...,...,...,...,...
2910,Twitter for iPhone,RT @DonaldJTrumpJr: Great group at our Victory...,10/06/2016 18:41,3391,0,True,7.840000e+17
2911,Twitter for iPhone,Volunteer to be a Trump Election Observer. Sig...,10/06/2016 20:45,7809,13830,False,7.840000e+17
2912,Twitter for iPhone,'Donald Trump: A President for All Americans' ...,10/06/2016 20:47,10933,23522,False,7.840000e+17
2913,Twitter for iPhone,VOTE #TrumpPence16 on 11/8/16! https://t.co/12...,10/06/2016 21:51,9062,19862,False,7.840000e+17


In [9]:
df_train["normalized_completion"] = df_train["source"].apply(lambda x: 1 if x=="Twitter for Android" else 0)

/tmp/ipykernel_569319/2754263311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["normalized_completion"] = df_train["source"].apply(lambda x: 1 if x=="Twitter for Android" else 0)


In [10]:
df_train

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,normalized_completion
0,Twitter for Android,I will be on @FoxNews live with members of my...,01/01/2016 00:44,2108,6735,False,6.830000e+17,1
1,Twitter for iPhone,HAPPY NEW YEAR &amp; THANK YOU! https://t.co/Y...,01/01/2016 03:25,3460,8581,False,6.830000e+17,0
2,Twitter for iPhone,#HappyNewYearAmerica! https://t.co/EeQb8PDrUe,01/01/2016 05:18,3434,9143,False,6.830000e+17,0
3,Twitter for iPhone,Happy New Year from #MarALago! Thank you to my...,01/01/2016 06:07,1948,8258,False,6.830000e+17,0
4,Twitter for Android,Well the year has officially begun. I have man...,01/01/2016 23:00,2642,8495,False,6.830000e+17,1
...,...,...,...,...,...,...,...,...
2910,Twitter for iPhone,RT @DonaldJTrumpJr: Great group at our Victory...,10/06/2016 18:41,3391,0,True,7.840000e+17,0
2911,Twitter for iPhone,Volunteer to be a Trump Election Observer. Sig...,10/06/2016 20:45,7809,13830,False,7.840000e+17,0
2912,Twitter for iPhone,'Donald Trump: A President for All Americans' ...,10/06/2016 20:47,10933,23522,False,7.840000e+17,0
2913,Twitter for iPhone,VOTE #TrumpPence16 on 11/8/16! https://t.co/12...,10/06/2016 21:51,9062,19862,False,7.840000e+17,0


In [11]:
df_train["normalized_completion"].value_counts()

normalized_completion
0    1541
1    1059
Name: count, dtype: int64

In [12]:
dict_train = df_train.to_dict(orient = "list")
train_dataset = Dataset.from_dict(dict_train)

In [13]:
train_dataset = train_dataset.remove_columns(["source","created_at","retweet_count","favorite_count","is_retweet","id_str"])
train_dataset = train_dataset.rename_column("normalized_completion", "label")
train_dataset = train_dataset.rename_column("text", "prompt")

In [14]:
train_dataset[0]

{'prompt': 'I will be on @FoxNews live  with members of my family at 11:50 P.M. We will ring in the New Year together! MAKE AMERICA GREAT AGAIN!',
 'label': 1}

In [15]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 107kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 3.35MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 545kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 750kB/s]


In [16]:
def preprocess_function(examples):
    return tokenizer(examples["prompt"], truncation=True)
 
tokenized_train = train_dataset.map(preprocess_function, batched=True)
# tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 2600/2600 [00:00<00:00, 17788.91 examples/s]


In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


model.safetensors: 100%|██████████| 268M/268M [01:16<00:00, 3.52MB/s] 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
  
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [66]:
from transformers import TrainingArguments, Trainer
  
training_args = TrainingArguments(
   output_dir="./deberta",
   learning_rate=2e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=7,
   weight_decay=0.01,
   save_strategy="steps",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_train,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [67]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.130000
1000,0.068500
1500,0.045300
2000,0.042400
2500,0.021000
3000,0.020000
3500,0.015300
4000,0.012900
4500,0.010400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4550, training_loss=0.040176393151754045, metrics={'train_runtime': 1069.3417, 'train_samples_per_second': 17.02, 'train_steps_per_second': 4.255, 'total_flos': 208318438106736.0, 'train_loss': 0.040176393151754045, 'epoch': 7.0})

In [68]:
trainer.save_model("deberat_models/ak")

# Inference

In [50]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel
import numpy as np
import re
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [69]:
MODEL = "deberat_models/ak"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)
model = model.to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [71]:
#prompt = "Redact PII with PII type in the text: Sarah Taylors Employee Identification Number is 987654322 whose home address is 91011 Oak Street, Anytown. Sarah Taylors Employee Identification Number is 987654322 whose home address is 91011 Oak Street, Anytown. Sarah Taylors Employee Identification Number is 987654322 whose home address is 91011 Oak Street, Anytown. Sarah Taylors Employee Identification Number is 987654322 whose home address is 91011 Oak Street, Anytown. ->"
def infer(text):
    inputs = tokenizer(text, return_tensors="pt").input_ids.cuda()
    with torch.no_grad():
        logits = model(inputs).logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

In [48]:
text = 'I will be on @FoxNews live  with members of my family at 11:50 P.M. We will ring in the New Year together! MAKE AMERICA GREAT AGAIN!'

In [73]:
all_results = []
for i in df_test["text"]:
    all_results.append(infer(i))

In [74]:
gt = [0 if i == 'Twitter for iPhone' else 1 for i in df_test["source"] ]

In [75]:
accuracy_score(gt,all_results)

0.9248

In [76]:
precision_score(gt,all_results)

0.9004739336492891

In [77]:
recall_score(gt, all_results)

0.8796296296296297